<h2>PRÁCTICA 2 - FUNDAMENTOS DE APRENDIZAJE AUTOMÁTICO</h1>
<h3>Realizada la práctica por:<br/>
    <ol>
    - Pablo Díez del Pozo<br/>
    - Alejandro Alcalá Álvarez<br/>
   </ol>
 </h3>


<h3>Código</h3>

En las siguientes celdas se inserta todo el código desarrollado para poder completar el funcionamiento requerido de la práctica.

<p>La clase <b>Datos</b> es idéntica a la que desarrollamos para la <b>Práctica 0</b>, con dos funciones añadidas, muy importantes para el correcto funcionamiento de el clasificador de <b>Vecinos Próximos</b>. Estas funciones son las siguientes:</p>
<ol> 
    <li><b>calcularMediasDesv(self, datos):</b> esta función permite calcular las medias y desviaciones típicas de cada uno de los atributos del dataset.3
    <li><b>normalizarDatos(self, datos):</b> llama a la función anterior y utiliza los valores que hemos calculado para normalizar el dataset, únicamente los valores continuos, ya que los discretos no es necesario normalizarlos
       

In [3]:
import numpy as np

class Datos:
    TiposDeAtributos = ('Continuo', 'Nominal')

    # TODO: procesar el fichero para asignar correctamente las variables tipoAtributos, nombreAtributos, nominalAtributos, datos y diccionarios
    # NOTA: No confundir TiposDeAtributos con tipoAtributos
    def __init__(self, nombreFichero):

        with open(nombreFichero, "r") as f:
            # Guardamos el numero de datos que contiene el DataSet y esta en la primera linea
            self.numDatos = int(f.readline())

            # Guardamos el nombre de los atributos
            self.nombreAtributos = f.readline().strip('\n').split(',')
            # print(self.nombreAtributos)

            # Leemos el tipo de los atributos de las variables y eliminamos el ultimo que es un salto de linea
            self.tipoAtributos = f.readline().strip('\n').split(',')
            # print(self.tipoAtributos)

            # Comprobamos que todos los atributos sean Continuos o Nominales
            if any(atr not in Datos.TiposDeAtributos for atr in self.tipoAtributos):
                raise ValueError("Tipo de atributo erroneo")

            # Segun el atributo, asignamos True o False.
            self.nominalAtributos = []

            # Guardamos en la lista nominalAtributos en la posicion de cada uno si es o no Nominal
            for tipo in self.tipoAtributos:
                if tipo == self.TiposDeAtributos[0]:
                    self.nominalAtributos.append(False)
                else:
                    self.nominalAtributos.append(True)


            # Guardamos los datos del fichero y los formateamos, de tal forma que cada linea es una lista
            datos = f.readlines()
            datosFormat = []
            for lista in datos:
                datosFormat.append(lista.strip('\n').split(','))

            listaDatosAtributos = []
            for i in range(len(self.tipoAtributos)):
                listaDatosAtributos.append([])

            # Hacemos la traspuesta de los datos que guardamos para que cada lista de atributo guarde todos los datos
            # de cada atributo.
            for lista in datosFormat:
                i = 0
                for item in lista:
                    listaDatosAtributos[i].append(item)
                    i += 1

            # Ordenamos y hacemos un set para eliminar repetidos.
            i = 0
            for item in listaDatosAtributos:
                listaDatosAtributos[i] = sorted(set(item))
                i += 1

            # Creacion de lista diccionarios, en caso de que el atributo sea Continuo, el diccionario estara vacio
            self.listaDicts = []
            for i in range(len(self.tipoAtributos)):
                self.listaDicts.append({})

            # Creamos el diccionario tal y como se describe en las diapositivas, por orden y asignando valores numericos crecientes
            i = 0
            for atributo in listaDatosAtributos:
                k = 0
                if self.tipoAtributos[i] == "Nominal":
                    for dato in atributo:
                        self.listaDicts[i][dato] = k
                        k += 1
                i += 1

            # Creacion de la matriz de datos utilizando el diccionario para mapear los valores
            # En primer lugar, creamos una matriz vacia de tamaña numero de atributos.
            self.datos = np.empty((int(self.numDatos), int(len(self.tipoAtributos))))
            i = 0
            j = 0

            # Metemos los datos en la matriz, mapeando con los diccionarios en el caso de que sean Nominales, y si son continuos normal.
            for i in range(int(self.numDatos)):
                for j in range(len(self.tipoAtributos)):
                    if self.tipoAtributos[j] == 'Nominal':
                        self.datos[i][j] = self.listaDicts[j].get(str(datosFormat[i][j]))
                    else:
                        self.datos[i][j] = datosFormat[i][j]

            f.close()

    # TODO: implementar en la practica 1
    def extraeDatos(self, idx):
        return self.datos[idx]


    def calcularMediasDesv(self,datos):

        # datosTrain es la matriz directamente, no hay que extraer nada
        # Se calcula la desv. tipica y la media para los atributos continuos

        # Creamos una lista con los indices que son continuos para calcular la media y desv tipica sobre ellos.
        self.indicesContinuos = []
        for i in range(len(self.nominalAtributos)):
            if self.nominalAtributos[i] == False:
                self.indicesContinuos.append(i)

        # Matriz que guarda el valor por atributo de la desv. tipica y la media
        estadisticas = np.zeros((2,len(self.nominalAtributos)))

        # Calculamos la desv tipica y media de los indices continuos sobre todos los datos que recibimos por param.
        for i in self.indicesContinuos:
            estadisticas[0,i] = np.mean(datos[:,i])
            estadisticas[1,i] = np.std(datos[:,i])

        return estadisticas


    def normalizarDatos(self, datos):

        estadisticas = self.calcularMediasDesv(datos)

        # Normalizamos los datos
        for i in self.indicesContinuos:
            datos[:,i] -= estadisticas[0,i]
            datos[:,i] /= estadisticas[1,i]

        return datos, estadisticas

La siguiente clase, <b>EstrategiaParticionado</b>, es exactamente igual que en la práctica anterior:

In [4]:
from abc import ABCMeta,abstractmethod

from abc import ABCMeta, abstractmethod
import numpy as np


class Particion():

  # Esta clase mantiene la lista de �ndices de Train y Test para cada partici�n del conjunto de particiones
  def __init__(self, train=[], test=[]):
    self.indicesTrain = train
    self.indicesTest = test

  def __str__(self):
    return "Train: {}\nTest:  {}".format(str(self.indicesTrain), str(self.indicesTest))


class EstrategiaParticionado:
  # Clase abstracta
  __metaclass__ = ABCMeta

  # Lista de las particiones
  def __init__(self, nombre=""):
    self.nombreEstrategia = nombre
    self.numeroParticiones = 0
    self.particiones = []

  # Atributos: deben rellenarse adecuadamente para cada estrategia concreta: nombreEstrategia, numeroParticiones, listaParticiones. Se pasan en el constructor

  @abstractmethod
  # TODO: esta funcion deben ser implementadas en cada estrategia concreta
  def creaParticiones(self, datos, seed=None):
    pass


class ValidacionSimple(EstrategiaParticionado):

  def __init__(self, porcentaje):
    self.porcentaje = porcentaje
    super().__init__("Validacion simple")

  # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
  # Devuelve una lista de particiones (clase Particion)
  # TODO: implementar
  def creaParticiones(self, datos, seed=None):
    np.random.seed(seed)
    self.numeroParticiones = 1

    # Generamos una lista con todos los números de datos aleatorios
    indicesAleatorios = np.random.permutation(int(datos.numDatos))

    # Creamos la particion, en funcion del porcentaje especificado
    self.particiones = [Particion(indicesAleatorios[:int(datos.numDatos * self.porcentaje)],
                                  indicesAleatorios[int(datos.numDatos * self.porcentaje):])]

    return self.particiones


class ValidacionCruzada(EstrategiaParticionado):

  # Crea particiones segun el metodo de validacion cruzada.
  # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
  # Esta funcion devuelve una lista de particiones (clase Particion)
  # TODO: implementar

  def __init__(self, k):
    self.k = k
    super().__init__("Validacion cruzada")

  def creaParticiones(self, datos, seed=None):

    np.random.seed(seed)

    self.numeroParticiones = self.k

    # Generamos una lista con todos los números de datos aleatorios
    indicesAleatorios = np.random.permutation(int(datos.numDatos))

    # Hallamos el tamaño de cada bloque
    tamBloque = int(datos.numDatos / self.k)

    datosSobran = datos.numDatos - (tamBloque * self.k)
    count = 0
    for i in range(self.k):

      train = np.delete(indicesAleatorios, range(i * tamBloque, (i + 1) * tamBloque))
      test = indicesAleatorios[i * tamBloque:(i + 1) * tamBloque]

      # Caso en el que la cuenta es justa
      if datosSobran == 0:
        self.particiones.append(Particion(train, test))

      # Contemplamos el caso de que la division para sacar el numero de subconjuntos no fuese entera
      if datosSobran > 0:
        count += 1
        particionTest = np.append(test, train[(datos.numDatos - tamBloque) - i - 1])
        particionTrain = np.delete(train, (datos.numDatos - tamBloque) - i - 1)
        datosSobran -= 1
        self.particiones.append(Particion(particionTrain, particionTest))

A continuación, tenemos la clase <b>Clasificador</b>, junto con <b>ClasificadorNaiveBayes</b>, idénticas a las de la práctica 1:

In [5]:
from abc import ABCMeta,abstractmethod
import numpy as np
import math
from collections import Counter
from sortedcontainers import SortedDict
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from numpy import linalg as LA
from statistics import mode

class Clasificador:
  # Clase abstracta
  __metaclass__ = ABCMeta

  # Metodos abstractos que se implementan en casa clasificador concreto
  @abstractmethod
  # TODO: esta funcion debe ser implementada en cada clasificador concreto
  # datosTrain: matriz numpy con los datos de entrenamiento
  # atributosDiscretos: array bool con la indicatriz de los atributos nominales
  # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion de variables discretas
  def entrenamiento(self, datos, datosTrain, atributosDiscretos, diccionario):
    pass

  @abstractmethod
  # TODO: esta funcion debe ser implementada en cada clasificador concreto
  # devuelve un numpy array con las predicciones
  def clasifica(self, datosTest, atributosDiscretos, diccionario):
    pass

  # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
  # TODO: implementar
  def error(self, datos, pred):
    # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error
    i = 0
    real = datos[:, -1]
    error = 0
    for i in range(len(real)):
      if real[i] != pred[i]:
        error += 1
    err = (error) / (len(real) + 0.0)
    return err

  # Realiza una clasificacion utilizando una estrategia de particionado determinada
  # particionado : estrategia de validacion que queremos utilizar
  # dataset : clase de tipo Datos que utilizamos para entrenar y clasificar el modelo
  # clasificador: instancia del clasificador que se va a usar
  # TODO: implementar esta funcion
  def validacion(self, particionado, dataset, clasificador, seed: object = None):

    # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
    # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
    # y obtenemos el error en la particion de test i
    # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
    # y obtenemos el error en la particion test. Otra opci�n es repetir la validaci�n simple un n�mero especificado de veces, obteniendo en cada una un error. Finalmente se calcular�a la media.
    errores = 0
    # particionado.creaParticiones(dataset, seed)
    # Comprobamos si es por validación cruzada o simple, por la longitud de la lista de particiones
    particionado.particiones = []
    particionado.creaParticiones(dataset)

    # Validación Simple
    if len(particionado.particiones) == 1:
      clasificador.entrenamiento(dataset, particionado.particiones[0].indicesTrain)
      pred = clasificador.clasifica(dataset.datos, particionado.particiones[0].indicesTest)
      ret = self.error(dataset.extraeDatos(particionado.particiones[0].indicesTest), pred)
      if ret > 0:
        return ret
      else:
        return 0

    # Validación Cruzada
    else:
      lista_error = []
      for particion in particionado.particiones:
        clasificador.entrenamiento(dataset, particion.indicesTrain)
        pred = clasificador.clasifica(dataset.datos, particion.indicesTest)
        ret = self.error(dataset.extraeDatos(particion.indicesTest), pred)
        lista_error.append(ret)
      return np.mean(lista_error), np.std(lista_error)

  def matrizConfusion(self, dataset, datosTest, prediccion):

    # Calculamos la matriz de confusion utlizando sk-learn. Solo se calcula en el caso de que la clasificacion sea binaria.
    testData = dataset.extraeDatos(datosTest)
    clase_real = testData[:, -1]

    matriz = confusion_matrix(prediccion, clase_real)

    # La funcion ravel() devuelve todas las estadisticas relacionadas con la matriz de confusion
    tn, fp, fn, tp = matriz.ravel()

    # Calculamos las tasas extraídas de la matriz de confusión
    tpr = tp / (tp + fn)
    fpr = fp / (fp + fn)

    self.lista_tpr.append(tpr)
    self.lista_fpr.append(fpr)

    return matriz

  def curvaROC(self):

    x = np.linspace(0, 1, 100)
    plt.plot(x, x, c='blue')
    for i in range(len(self.lista_fpr)):
      plt.plot(self.lista_fpr[i], self.lista_tpr[i], 'ro')
    plt.show()

class ClasificadorNaiveBayes(Clasificador):

  def __init__(self, laplace):
    self.laplace = laplace
    self.lista_fpr = []
    self.lista_tpr = []

  def entrenamiento(self, dataset, datosTrain):

    # Cargamos todos los datos de la clase del dataset desde la matriz de datos
    clasesTrain = dataset.extraeDatos(datosTrain)
    self.numClases = clasesTrain[:, -1]

    # Contamos las apariciones de cada uno para luego calcular la probabilidad a priori de cada clase
    counter = Counter(self.numClases)

    # Calculamos la probabilidad de la clase y lo metemos en un diccionario ordenado segun el numero
    # correspondiente a cada clase asignado en el diccionario
    self.dictPrioris = {}
    for k in counter:
      k = int(k)
      counter[k] = counter[k] / len(self.numClases)
      self.dictPrioris[k] = counter[k]

    # Aqui ordenamos el diccionario para que esten en el mismo orden de como extraemos los datos del dataset
    self.dictPrioris = SortedDict(self.dictPrioris)

    # Calcular tablas de probabilidades del entrenamiento. Tenemos que calcular por cada atributo una cuenta
    # de las apariciones en cada clase
    # Creamos una lista de matrices, donde vamos almacenar todos los datos que hemos obtenido en los datos de Test
    self.posteriori = np.zeros(len(dataset.nombreAtributos) - 1, dtype=object)

    # Recorremos todos los datos de la matriz sin llegar a la clase
    for i in range(len(dataset.nombreAtributos) - 1):

      # Si el dato que obtenemos es Nominal haremos el recuento de todas las veces que sale la P(D|H)
      if dataset.nominalAtributos[i] == True:

        # Creamos una matriz de tamaño X: Número de Atributos menos la clase Y: Número de clases
        post = np.zeros((len(dataset.listaDicts[i]), len(dataset.listaDicts[-1])))

        # Aqui contamos todos las datos que queremos del datos Train para construir la matriz de entrenamiento
        for c in range(len(dataset.listaDicts[-1])):
          datosEnt = dataset.extraeDatos(datosTrain)
          dat = datosEnt[:, i]
          repes = Counter(dat[datosEnt[:, -1] == c])
          for r in repes:
            post[int(r), c] = repes[r]
          if self.laplace == True:
            self.posteriori[i] = post + 1
          else:
            self.posteriori[i] = post

      # Si el dato es Continuo obtendremos la media y la desviación tipica de la clase
      else:

        # Creamos una matriz de X: Los datos de Media y Desivación típica Y: Número de clases
        post = np.zeros((2, len(dataset.listaDicts[-1])))

        # Aqui obtenemos la media y desviación tipica de cada clase, despues de tener los datos de entrenamiento
        for c in range(len(dataset.listaDicts[-1])):
          datosEnt = dataset.extraeDatos(datosTrain)
          dat = datosEnt[:, i]
          datos = dat[datosEnt[:, -1] == c]
          post[0][c] = np.mean(datos)
          post[1][c] = np.std(datos)
        self.posteriori[i] = post


    # Calculamos los valores de los posteriori de todos las tablas anteriores
    for i in range(len(dataset.listaDicts) - 1):
      if dataset.nominalAtributos[i] == True:
        self.posteriori[i] /= sum(self.posteriori[i])

  def clasifica(self, dataset, datosTest):
    acum_probs = 1
    self.prediccion = []
    datTest = dataset[datosTest]

    # Ahora vamos a estudiar la probabilidad de la clase con los datos obtenidos en el entrenamiento
    # Recorremos todos las datos de la matriz de los datos Test
    for dato in datTest:
      mapa = []
      # Aqui obtenemos los prioris de cada clase para poder obtener la probabilidad de cada una
      for clase in range(len(self.dictPrioris)):
        listaVerosimilitudes = []
        # Aqui obtenemos cada valor posteriori de nuestro entrenamiento de los datos, es decir, P(D|H)
        for atributo in range(len(self.posteriori)):
          if dataset.nominalAtributos[atributo] == True:
            prob = self.posteriori[atributo][int(dato[atributo])][clase]
            listaVerosimilitudes.append(prob)

          # Aqui obtenemos la probabilidad de los atibutos continuos
          else:
            # Hacemos la formula de la distribucion normal
            exp1 = 1 / (self.posteriori[atributo][1][clase] * math.sqrt(2 * math.pi))
            exp2 = np.power((dato[atributo] - self.posteriori[atributo][0][clase]), 2)
            exp3 = np.power(self.posteriori[atributo][1][clase], 2)
            exp4 = exp2 / exp3
            exp4 = math.exp((-1 / 2) * exp4)
            prob = exp1 * exp4
            listaVerosimilitudes.append(prob)

        for verosimilitud in listaVerosimilitudes:
          acum_probs *= verosimilitud
        acum_probs *= self.dictPrioris.get(clase)
        mapa.append(acum_probs)
        acum_probs = 1

      # Aqui obtenemos la predicción de mayor probabilidad y la guardamos en nuestra lista de predicciones
      self.prediccion.append(np.argmax(mapa))


    # Devolvemos la lista con la predicción de nuestro clasifica
    return self.prediccion

En las siguientes celdas aparecen las clases <b>ClasificadorVecinosProximos</b> y <b>ClasificadorRegresionLogistica</b>, ambas necesarias para la funcionalidad requerida en esta práctica.

La clase <b>ClasificadorVecinosProximos</b> recibe como parámetros en su constructor tanto el valor de <b>K</b>y si se quiere normalizar el dataset o no, por defecto la normalización está activada. El método <b>entrenamiento()</b>, normaliza el dataset si tenemos esa opción a True y guarda los datos de entrenamiento. En el caso de que no queremos normalizar, simplemente guarda los datos de entrenamiento. El método <b>clasifica()</b> implementa el algoritmo de <b>Vecinos Próximos</b>, calculando las distancias de los datos de Test con todos los datos de Train y quedandose con la clase mayoritaría de los k elementos con menor distancia, para cada dato del conjunto de Test. Todo lo anterior genera una lista con las clases predichas, que es lo que devuelve la función.

La clase <b>ClasificadorRegresionLogistica</b> se inicializa con dos parámetros, la <b>Constante de Aprendizaje</b>, que se utiliza para actualizar la función de pesos del método <b>entrenamiento()</b>, y el <b>número de épocas</b>. Además, cuenta con una función auxiliar <b>sigmoidal()</b> que ejecuta la sigmoidal del valor dado por el producto escalar de la función de pesos <b>(w)</b> y el vector compuesto por el dato de entrenamiento <b>(x)</b> en el que nos encontramos. La función <b>entrenamiento()</b> utiliza el algoritmo de la Regresión Logistica hasta el número de épocas definido, obteniendo el vector de pesos final, que posteriormente se utiliza en <b>clasifica()</b>, que genera una predicción para cada dato del test, aplicando la sigmoidal del dato multiplicado por el vector de pesos final que obtenemos al entrenar el modelo, si se obtiene un valor mayor de 0.5 el dato es de clase 1, si no de clase 2. 

<h4>Particularidades Vecinos Próximos</h4>

No hemos realizado distinción entre el caso de que el atributo sea discreto o continuo para calcular las distancias, ya que nuestra idea era utilizar distancia Euclídea en el caso de los continuos y distancia Manhattan en el caso de los discretos, pero como codificamos el dataset previamente en formato numérico, esas distancias no se ven afectadas realizando la distinción, ya que son proporcionales.

El caso en el que el atributo tiene un orden natural, como el que se describe en el enunciado con el caso de los meses, también lo hemos obviado, ya que la codificación del dataset a numéricos resuelve este problema. Por ejemplo, el orden natural de los meses, en nuestro sería del 0 al 11 y en nuestro caso codificamos los meses según esa notación. El único posible problema es que se codifican en orden de lectura, es decir, si aparece primero Noviembre y luego Junio, esos serían el 0 y el 1. Esta situación no es ningún problema, ya que las distancias se mantienen costantes en todos los casos para ese nuevo orden, siendo proporcionales en todos los casos

In [ ]:
class ClasificadorVecinosProximos(Clasificador):

  def __init__(self, k, normaliza = True):

    # k define el numero de vecinos y normaliza si queremos normalizar los datos o no.
    self.k = k
    self.normaliza = normaliza
    self.lista_fpr = []
    self.lista_tpr = []
    super().__init__()

  def entrenamiento(self, dataset, datosTrain):

    # Normalizar los datos en caso de que la variable sea True
    if self.normaliza == True:
      self.datosClasifica, _ = dataset.normalizarDatos(dataset.datos)
    else:
      self.datosClasifica = dataset.datos

    # Datos de train
    self.datTrain = self.datosClasifica[datosTrain]

    return self.datTrain

  def clasifica(self, dataset, datosTest = None):

    if datosTest is None:
      datTest = dataset[range(len(dataset))]
    else:
      datTest = self.datosClasifica[datosTest]

    prediccion = []

    # Cogemos cada ejemplo en datosTest para calcular la distancia a cada uno de los ejemplos de Train
    for datoTest in datTest[:, :-1]:
      distancias = []
      for datoTrain in self.datTrain[:, :-1]:
        sumatorio_dist_euclideas = 0
        for i in range(len(datoTest)):
          sumatorio_dist_euclideas += LA.linalg.norm(datoTest[i] - datoTrain[i])
        distancias.append(sumatorio_dist_euclideas)

      k_datos = self.datTrain[np.argsort(distancias)[:self.k]]
      prediccion.append(mode(k_datos[:,-1]))

    return prediccion

In [6]:
# Clase que define un clasificador utilizando el metodo de Regresion Logistica
class ClasificadorRegresionLogistica(Clasificador):

  def __init__(self, constante_aprendizaje, epocas):
    # Se utiliza para inicializar la constante de aprendizaje y el numero de epocas necesarias para el train
    self.constante_aprendizaje = constante_aprendizaje
    self.epocas = epocas

    super().__init__()

  # Funcion que realiza el calculo de la sigmoidal de el numero que recibe por parámetro
  def sigmoidal(self, a):

    if a >= 100:
      return 1
    elif a < 100:
      return 0
    else:
      return 1.0 / (1.0 + math.exp(-a))

  def entrenamiento(self, dataset, datosTrain):

    # Datos de entrenamiento
    self.datTrain = dataset.extraeDatos(datosTrain)

    # Inicializamos el vector w
    self.vector_w = np.random.uniform(low=-0.5, high=0.5, size=len(dataset.tipoAtributos))

    i = 0

    # Algoritmo del entrenamiento de regresion lineal
    while i < self.epocas:
      for dato in self.datTrain:

        # Calculamos vector x
        x = np.append([1],dato[:-1])

        # Calculamos w.x
        wx = np.dot(self.vector_w,x)

        # Sigmoidal de w.x
        sigmoidal = self.sigmoidal(wx)

        # Actualizamos el vector w
        self.vector_w = self.vector_w - (np.dot(self.constante_aprendizaje * (sigmoidal - dato[-1]),x))

      i += 1

    return self.vector_w

  def clasifica(self, dataset, datosTest = None):

    if datosTest is None:
      datosTest = range(len(dataset))

    datTest = dataset[datosTest]
    prediccion = []

    for dato in datTest:

      # Vector x
      x = np.append([1], dato[:-1])

      # Calculo de la sigmoidal que nos da la probabilidad de que el dato sea de clase 1
      wx = np.dot(self.vector_w, x)
      sigmoidal = self.sigmoidal(wx)

      # Prediccion del clasificador
      if(sigmoidal >= 0.5):
        prediccion.append(1)
      else:
        prediccion.append(0)

    return prediccion

Una vez explicadas las clases creadas, generamos los datasets y las particiones que vamos a utilizar para la extracción de resultados de los mismos.

In [7]:
# Cargamos todos los datasets

dataset_example1 =Datos('../Datasets/example1.data')
dataset_example2 =Datos('../Datasets/example2.data')
dataset_example3 =Datos('../Datasets/example3.data')
dataset_example4 = Datos('../Datasets/example4.data')
dataset_online = Datos('../Datasets/online_shoppers.data')
dataset_wdbc = Datos('../Datasets/wdbc.data')

In [8]:
# Validacion simple

VS_example1 = ValidacionSimple(0.7)
VS_example2 = ValidacionSimple(0.7)
VS_example3 = ValidacionSimple(0.7)
VS_example4 = ValidacionSimple(0.7)
VS_online = ValidacionSimple(0.7)
VS_wdbc = ValidacionSimple(0.7)

VS_example1.creaParticiones(dataset_example1)
VS_example2.creaParticiones(dataset_example2)
VS_example3.creaParticiones(dataset_example3)
VS_example4.creaParticiones(dataset_example4)
VS_online.creaParticiones(dataset_online)
VS_wdbc.creaParticiones(dataset_wdbc)

VC_example1 = ValidacionCruzada(5)
VC_example2 = ValidacionCruzada(5)
VC_example3 = ValidacionCruzada(5)
VC_example4 = ValidacionCruzada(5)
VC_online = ValidacionCruzada(5)
VC_wdbc = ValidacionCruzada(5)

<h3>Apartado 1</h3>

En este apartado se detallan los resultados obtenidos utlizando la implementación propia de <b>Vecinos Próximos</b>, tanto normalizando los datos como sin hacerlo. Se recoge una tabla que muestra el rendimiento para los datasets de <b>wdbc.data</b> y <b>online_shoppers.data</b>.

En el caso del dataset <b>online_shoppers.data</b> solamente hemos utilizado validación simple, ya que tardaba mucho en ejecutarse, en el caso de <b>wdbc.data</b> si se ha utilizado tanto la validación simple como la cruzada, para observarlas diferencias.

<h4>Ejecuciones con Validación Simple y Online_Shoppers.data</h4>

In [ ]:
knn = ClasificadorVecinosProximos(1)
error = knn.validacion(VS_online, dataset_online, knn)

print("-- Validacion simple -- Dataset Online_Shoppers -- K: 1 -- Error:",error)

In [ ]:
knn = ClasificadorVecinosProximos(3)
error = knn.validacion(VS_online, dataset_online, knn)

print("-- Validacion simple -- Dataset Online_Shoppers -- K: 3 -- Error:",error)

In [ ]:
knn = ClasificadorVecinosProximos(5)
error = knn.validacion(VS_online, dataset_online, knn)

print("-- Validacion simple -- Dataset Online_Shoppers -- K: 5 -- Error:",error)

In [ ]:
knn = ClasificadorVecinosProximos(11)
error = knn.validacion(VS_online, dataset_online, knn)

print("-- Validacion simple -- Dataset Online_Shoppers -- K: 11 -- Error:",error)

<h4>Ejecuciones con Validación Simple y WDBC.data</h4>

In [ ]:
knn = ClasificadorVecinosProximos(1)
error = knn.validacion(VS_online, dataset_online, knn)

print("-- Validacion simple -- Dataset Online_Shoppers -- K: 1 -- Error:",error)

In [ ]:
knn = ClasificadorVecinosProximos(3)
error = knn.validacion(VS_online, dataset_online, knn)

print("-- Validacion simple -- Dataset Online_Shoppers -- K: 3 -- Error:",error)

In [ ]:
knn = ClasificadorVecinosProximos(5)
error = knn.validacion(VS_online, dataset_online, knn)

print("-- Validacion simple -- Dataset Online_Shoppers -- K: 5 -- Error:",error)

In [ ]:
knn = ClasificadorVecinosProximos(11)
error = knn.validacion(VS_online, dataset_online, knn)

print("-- Validacion simple -- Dataset Online_Shoppers -- K: 11 -- Error:",error)

<h3>Apartado 2</h3>

En este apartado se muestra la ejecución, para los mismos datasets que en el apartado 1, del modelo de <b>Regresión Logísitca</b> que hemos implementado.

<h4>Ejecuciones con Validación Simple y Online_Shoppers.data</h4>

In [9]:
regresion_logistica = ClasificadorRegresionLogistica(0.1,10)
error = regresion_logistica.validacion(VS_online, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 10 -- n: 0.1 -- Error:",error)

-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 10 -- n: 0.1 -- Error: 0.11300351446336848


In [10]:
regresion_logistica = ClasificadorRegresionLogistica(1,10)
error = regresion_logistica.validacion(VS_online, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 10 -- n: 1 -- Error:",error)

-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 10 -- n: 1 -- Error: 0.14706677480400107


In [11]:
regresion_logistica = ClasificadorRegresionLogistica(0.01,10)
error = regresion_logistica.validacion(VS_online, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 10 -- n: 0.01 -- Error:",error)

-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 10 -- n: 0.01 -- Error: 0.5466342254663422


In [12]:
regresion_logistica = ClasificadorRegresionLogistica(0.1,100)
error = regresion_logistica.validacion(VS_online, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 100 -- n: 0.1 -- Error:",error)

-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 100 -- n: 0.1 -- Error: 0.12949445796161124


In [13]:
regresion_logistica = ClasificadorRegresionLogistica(1,100)
error = regresion_logistica.validacion(VS_online, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 100 -- n: 1 -- Error:",error)

-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 100 -- n: 1 -- Error: 0.5217626385509597


In [14]:
regresion_logistica = ClasificadorRegresionLogistica(0.01,100)
error = regresion_logistica.validacion(VS_online, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 100 -- n: 0.01 -- Error:",error)

-- Validacion simple -- Dataset Online_Shoppers -- Epocas: 100 -- n: 0.01 -- Error: 0.13760475804271424


<h4>Ejecuciones con Validación Simple y WDBC.data</h4>

In [15]:
regresion_logistica = ClasificadorRegresionLogistica(0.1,10)
error = regresion_logistica.validacion(VS_wdbc, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset WDBC -- Epocas: 10 -- n: 0.1 -- Error:",error)

-- Validacion simple -- Dataset WDBC -- Epocas: 10 -- n: 0.1 -- Error: 0.14841849148418493


In [16]:
regresion_logistica = ClasificadorRegresionLogistica(1,10)
error = regresion_logistica.validacion(VS_wdbc, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset WDBC -- Epocas: 10 -- n: 1 -- Error:",error)

-- Validacion simple -- Dataset WDBC -- Epocas: 10 -- n: 1 -- Error: 0.1300351446336848


In [17]:
regresion_logistica = ClasificadorRegresionLogistica(0.01,10)
error = regresion_logistica.validacion(VS_wdbc, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset WDBC -- Epocas: 10 -- n: 0.01 -- Error:",error)

-- Validacion simple -- Dataset WDBC -- Epocas: 10 -- n: 0.01 -- Error: 0.6623411732900784


In [18]:
regresion_logistica = ClasificadorRegresionLogistica(0.1,100)
error = regresion_logistica.validacion(VS_wdbc, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset WDBC -- Epocas: 100 -- n: 0.1 -- Error:",error)

-- Validacion simple -- Dataset WDBC -- Epocas: 100 -- n: 0.1 -- Error: 0.18140037848067045


In [19]:
regresion_logistica = ClasificadorRegresionLogistica(1,100)
error = regresion_logistica.validacion(VS_wdbc, dataset_online, regresion_logistica)

print("-- Validacion simple -- Dataset WDBC -- Epocas: 100 -- n: 1 -- Error:",error)

-- Validacion simple -- Dataset WDBC -- Epocas: 100 -- n: 1 -- Error: 0.12489862124898621


In [20]:
regresion_logistica = ClasificadorRegresionLogistica(0.01,100)
error = regresion_logistica.validacion(VS_wdbc, dataset_online, regresion_logistica)

print(" -- Validacion simple -- Dataset WDBC -- Epocas: 100 -- n: 0.01 -- Error:",error)

 -- Validacion simple -- Dataset WDBC -- Epocas: 100 -- n: 0.01 -- Error: 0.583941605839416


<h4>Ejecuciones con Validación Cruzada y Online_Shoppers.data</h4>

In [21]:
regresion_logistica = ClasificadorRegresionLogistica(0.1,10)
error, desv = regresion_logistica.validacion(VC_online, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 0.1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 0.1 --
    Error: 0.13341443633414435
    Desv. tipica 0.012530348007817755


In [22]:
regresion_logistica = ClasificadorRegresionLogistica(1,10)
error, desv = regresion_logistica.validacion(VC_online, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 0.1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 0.1 --
    Error: 0.11038118410381184
    Desv. tipica 0.004969172321829808


In [23]:
regresion_logistica = ClasificadorRegresionLogistica(0.01,10)
error, desv = regresion_logistica.validacion(VC_online, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 1 --
    Error: 0.25117599351175995
    Desv. tipica 0.14681874429950126


In [24]:
regresion_logistica = ClasificadorRegresionLogistica(0.1,100)
error, desv = regresion_logistica.validacion(VC_online, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 0.1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 0.1 --
    Error: 0.13909164639091648
    Desv. tipica 0.022145723491809904


In [25]:
regresion_logistica = ClasificadorRegresionLogistica(1,100)
error, desv = regresion_logistica.validacion(VC_online, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 1 --
    Error: 0.12384428223844282
    Desv. tipica 0.0052095658889705545


In [26]:
regresion_logistica = ClasificadorRegresionLogistica(0.01,100)
error, desv = regresion_logistica.validacion(VC_online, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 0.01 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 0.01 --
    Error: 0.14630981346309813
    Desv. tipica 0.04740010745196649


<h4>Ejecuciones con Validación Cruzada y WDBC.data</h4>

In [27]:
regresion_logistica = ClasificadorRegresionLogistica(0.1,10)
error, desv = regresion_logistica.validacion(VC_wdbc, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 0.1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 0.1 --
    Error: 0.12181670721816706
    Desv. tipica 0.00976137874399228


In [28]:
regresion_logistica = ClasificadorRegresionLogistica(1,10)
error, desv = regresion_logistica.validacion(VC_wdbc, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 1 --
    Error: 0.6055150040551501
    Desv. tipica 0.2454215399189112


In [29]:
regresion_logistica = ClasificadorRegresionLogistica(0.01,10)
error, desv = regresion_logistica.validacion(VC_wdbc, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 0.01 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 10 -- n: 0.01 --
    Error: 0.1280616382806164
    Desv. tipica 0.010550873880259153


In [30]:
regresion_logistica = ClasificadorRegresionLogistica(0.1,100)
error, desv = regresion_logistica.validacion(VC_wdbc, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 0.1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 0.1 --
    Error: 0.435198702351987
    Desv. tipica 0.16477854572784872


In [31]:
regresion_logistica = ClasificadorRegresionLogistica(1,100)
error, desv = regresion_logistica.validacion(VC_wdbc, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 1 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 1 --
    Error: 0.12222222222222223
    Desv. tipica 0.011557106472369634


In [32]:
regresion_logistica = ClasificadorRegresionLogistica(0.01,100)
error, desv = regresion_logistica.validacion(VC_wdbc, dataset_online, regresion_logistica)


print("-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 0.01 --")
print("    Error:", error)
print("    Desv. tipica", desv)

-- Validacion cruzada -- Dataset ONLINE_SHOPPERS -- Epocas: 100 -- n: 0.01 --
    Error: 0.16674776966747767
    Desv. tipica 0.09312390505951341


<h4>Resultados Regresión Logística</h4>

ONLINE_SHOPPERS

WDBC

<h3>Apartado 3</h3>

Ejecución de los algoritmos de SKLearn y de la representación de fronteras utilizando nuestros modelos.

In [33]:
import Clasificador
from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.pyplot as plt


# Autor Luis Lago y Manuel Sanchez Montanes
# Modificada por Gonzalo
def plotModel(x, y, clase, clf, title):
    x_min, x_max = x.min() - .2, x.max() + .2
    y_min, y_max = y.min() - .2, y.max() + .2

    hx = (x_max - x_min) / 100.
    hy = (y_max - y_min) / 100.

    xx, yy = np.meshgrid(np.arange(x_min, x_max, hx), np.arange(y_min, y_max, hy))

    if isinstance(clf, Clasificador.Clasificador):
        zeros = np.zeros(len(xx.ravel()))
        pred = clf.clasifica(np.c_[xx.ravel(), yy.ravel(), zeros])
        z = np.array(pred)
        # z = clf.clasifica(np.c_[xx.ravel(), yy.ravel()], [False, False, True], diccionarios)
    elif hasattr(clf, "decision_function"):
        z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

    z = z.reshape(xx.shape)
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    # ax = plt.subplot(1, 1, 1)
    plt.contourf(xx, yy, z, cmap=cm, alpha=.8)
    plt.contour(xx, yy, z, [0.5], linewidths=[2], colors=['k'])

    if clase is not None:
        plt.scatter(x[clase == 0], y[clase == 0], c='#FF0000')
        plt.scatter(x[clase == 1], y[clase == 1], c='#0000FF')
    else:
        plt.plot(x, y, 'g', linewidth=3)

    plt.gca().set_xlim(xx.min(), xx.max())
    plt.gca().set_ylim(yy.min(), yy.max())
    plt.grid(True)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title(title)
    plt.show()

<h4>Representación de fronteras Vecinos Próximos</h4>

Tomamos la K como 11 para todos los casos.

In [34]:
knn = ClasificadorVecinosProximos(11)
knn.entrenamiento(dataset, VS_example1.particiones[0].indicesTrain)
pred = knn.clasifica(dataset.datos,VS_example1.particiones[0].indicesTest)

x = dataset.datos[VS_example1.particiones[0].indicesTrain, 0]
y = dataset.datos[VS_example1.particiones[0].indicesTrain, 1]
clase = dataset.datos[VS_example1.particiones[0].indicesTrain, -1] != 0
plotModel(x, y, clase, knn, "-- Fronteras KNN -- K = 11 -- EXAMPLE1 --")

NameError: name 'ClasificadorVecinosProximos' is not defined

In [ ]:
knn = ClasificadorVecinosProximos(11)
knn.entrenamiento(dataset, VS_example2.particiones[0].indicesTrain)
pred = knn.clasifica(dataset.datos,VS_example2.particiones[0].indicesTest)

x = dataset.datos[VS_example2.particiones[0].indicesTrain, 0]
y = dataset.datos[VS_example2.particiones[0].indicesTrain, 1]
clase = dataset.datos[VS_example2.particiones[0].indicesTrain, -1] != 0
plotModel(x, y, clase, knn, "-- Fronteras KNN -- K = 11 -- EXAMPLE2 --")

In [ ]:
knn = ClasificadorVecinosProximos(11)
knn.entrenamiento(dataset, VS_example3.particiones[0].indicesTrain)
pred = knn.clasifica(dataset.datos,VS_example3.particiones[0].indicesTest)

x = dataset.datos[VS_example3.particiones[0].indicesTrain, 0]
y = dataset.datos[VS_example3.particiones[0].indicesTrain, 1]
clase = dataset.datos[VS_example3.particiones[0].indicesTrain, -1] != 0
plotModel(x, y, clase, knn, "-- Fronteras KNN -- K = 11 -- EXAMPLE3 --")

In [ ]:
knn = ClasificadorVecinosProximos(11)
knn.entrenamiento(dataset, VS_example4.particiones[0].indicesTrain)
pred = knn.clasifica(dataset.datos,VS_example4.particiones[0].indicesTest)

x = dataset.datos[VS_example4.particiones[0].indicesTrain, 0]
y = dataset.datos[VS_example4.particiones[0].indicesTrain, 1]
clase = dataset.datos[VS_example4.particiones[0].indicesTrain, -1] != 0
plotModel(x, y, clase, knn, "-- Fronteras KNN -- K = 11 -- EXAMPLE4 --")

<h4>Representación de fronteras Regresión Logística</h4>

In [ ]:
knn = ClasificadorVecinosProximos(11)
knn.entrenamiento(dataset, VS_example1.particiones[0].indicesTrain)
pred = knn.clasifica(dataset.datos,VS_example1.particiones[0].indicesTest)

x = dataset.datos[VS_example1.particiones[0].indicesTrain, 0]
y = dataset.datos[VS_example1.particiones[0].indicesTrain, 1]
clase = dataset.datos[VS_example1.particiones[0].indicesTrain, -1] != 0
plotModel(x, y, clase, knn, "-- Fronteras KNN -- K = 11 -- EXAMPLE1 --")

In [ ]:
knn = ClasificadorVecinosProximos(11)
knn.entrenamiento(dataset, VS_example2.particiones[0].indicesTrain)
pred = knn.clasifica(dataset.datos,VS_example2.particiones[0].indicesTest)

x = dataset.datos[VS_example2.particiones[0].indicesTrain, 0]
y = dataset.datos[VS_example2.particiones[0].indicesTrain, 1]
clase = dataset.datos[VS_example2.particiones[0].indicesTrain, -1] != 0
plotModel(x, y, clase, knn, "-- Fronteras KNN -- K = 11 -- EXAMPLE2 --")

In [ ]:
knn = ClasificadorVecinosProximos(11)
knn.entrenamiento(dataset, VS_example3.particiones[0].indicesTrain)
pred = knn.clasifica(dataset.datos,VS_example3.particiones[0].indicesTest)

x = dataset.datos[VS_example3.particiones[0].indicesTrain, 0]
y = dataset.datos[VS_example3.particiones[0].indicesTrain, 1]
clase = dataset.datos[VS_example3.particiones[0].indicesTrain, -1] != 0
plotModel(x, y, clase, knn, "-- Fronteras KNN -- K = 11 -- EXAMPLE3 --")

In [ ]:
knn = ClasificadorVecinosProximos(11)
knn.entrenamiento(dataset, VS_example4.particiones[0].indicesTrain)
pred = knn.clasifica(dataset.datos,VS_example4.particiones[0].indicesTest)

x = dataset.datos[VS_example4.particiones[0].indicesTrain, 0]
y = dataset.datos[VS_example4.particiones[0].indicesTrain, 1]
clase = dataset.datos[VS_example4.particiones[0].indicesTrain, -1] != 0
plotModel(x, y, clase, knn, "-- Fronteras KNN -- K = 11 -- EXAMPLE4 --")

<h3>Apartado 4</h3>

Comparación de Análisis ROC entre los 3 clasificadores implementados durante el curso para los dataset <b>wdbc.data</b> y <b>online_shoppers</b>.

In [ ]:
  logistic_reg = ClasificadorRegresionLogistica(0.5,250)
    logistic_reg.entrenamiento(dataset,estrategia.particiones[0].indicesTrain)
    pred = logistic_reg.clasifica(dataset.datos,estrategia.particiones[0].indicesTest)
    print(pred)
    error = logistic_reg.error(dataset.extraeDatos(estrategia.particiones[0].indicesTest),pred)
    print(error)
    matriz = logistic_reg.matrizConfusion(dataset,estrategia.particiones[0].indicesTest,pred)
    print(matriz)
    logistic_reg.curvaROC()